In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex2 import *
print("Setup Complete")

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


Setup Complete


In [3]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "openaq" dataset
dataset_ref = client.dataset("openaq", project="bigquery-public-data") #REFERENCE
# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)                              #FECTH

# Construct a reference to the "global_air_quality" table
table_ref = dataset_ref.table("global_air_quality")  #REFERENCE
# API request - fetch the table
table = client.get_table(table_ref)                  #FETCH

# Preview the first five lines of the "global_air_quality" table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  app.launch_new_instance()


,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours,location_geom
0,"Borówiec, ul. Drapałka",Borówiec,PL,bc,0.85217,2022-04-28 07:00:00+00:00,µg/m³,GIOS,1.0,52.276794,17.074114,POINT(52.276794 1)
1,"Kraków, ul. Bulwarowa",Kraków,PL,bc,0.91284,2022-04-27 23:00:00+00:00,µg/m³,GIOS,1.0,50.069308,20.053492,POINT(50.069308 1)
2,"Płock, ul. Reja",Płock,PL,bc,1.41000,2022-03-30 04:00:00+00:00,µg/m³,GIOS,1.0,52.550938,19.709791,POINT(52.550938 1)
3,"Elbląg, ul. Bażyńskiego",Elbląg,PL,bc,0.33607,2022-05-03 13:00:00+00:00,µg/m³,GIOS,1.0,54.167847,19.410942,POINT(54.167847 1)
4,"Piastów, ul. Pułaskiego",Piastów,PL,bc,0.51000,2022-05-11 05:00:00+00:00,µg/m³,GIOS,1.0,52.191728,20.837489,POINT(52.191728 1)


In [ ]:
print(dataset.description)
print('\nTables:')
print([table.table_id for table in list(client.list_tables(dataset))])
print('\nSchema:\n',table.schema)

# 1. Select Excercise

### 1. Which countries have reported pollution levels in units of "ppm"? In the code cell below, set first_query to an SQL query that pulls the appropriate entries from the country column.

In case it's useful to see an example query, here's some code from the tutorial:
```
query = """
        SELECT city
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """
```       

In [ ]:
query_1 = """ Select country
from `bigquery-public-data.openaq.global_air_quality`
where unit="ppm" """

In [ ]:
# Query to select countries with units of "ppm"
query = query_1
# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
first_results = query_job.to_dataframe()

# View top few rows of results
print(query_results.head())

In [ ]:
q_1.check()

### 2. Which pollution levels were reported to be exactly 0?

* Set zero_pollution_query to select all columns of the rows where the value column is 0.
* Set zero_pollution_results to a pandas DataFrame containing the query results.

In [ ]:
query_2 = """ Select *
from `bigquery-public-data.openaq.global_air_quality`
where value=0 """

In [ ]:
# Query to select countries with units of "ppm"
query = query_2
# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
zero_pollution_results = query_job.to_dataframe()

# View top few rows of results
print(zero_pollution_results.head())
# Check your answer
q_2.check()

# 2. Groupby Excercise

In [4]:
# Construct a reference to the "hacker_news" dataset
dataset_hack_ref = client.dataset("hacker_news", project="bigquery-public-data")
# API request - fetch the dataset
dataset_hack = client.get_dataset(dataset_hack_ref)

# Construct a reference to the "comments" table
table_com_ref = dataset_hack_ref.table("comments")
# API request - fetch the table
table_com = client.get_table(table_com_ref)

# Preview the first five lines of the "comments" table
df_head_table_com = client.list_rows(table_com, max_results=20).to_dataframe()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  if sys.path[0] == "":


In [5]:
df_head_table_com

,id,by,author,time,time_ts,text,parent,deleted,dead,ranking
0,2701393,5l,5l,1309184881,2011-06-27 14:28:01+00:00,And the glazier who fixed all the broken windo...,2701243,None,None,0
1,5811403,99,99,1370234048,2013-06-03 04:34:08+00:00,Does canada have the equivalent of H1B/Green c...,5804452,None,None,0
2,21623,AF,AF,1178992400,2007-05-12 17:53:20+00:00,"Speaking of Rails, there are other options in ...",21611,None,None,0
3,10159727,EA,EA,1441206574,2015-09-02 15:09:34+00:00,Humans and large livestock (and maybe even pet...,10159396,None,None,0
4,2988424,Iv,Iv,1315853580,2011-09-12 18:53:00+00:00,I must say I reacted in the same way when I re...,2988179,None,None,0
5,3867418,Iv,Iv,1334921984,2012-04-20 11:39:44+00:00,&#62; There's a whole class of 'mom &#38; pop'...,3867404,None,None,0
6,3925617,Iv,Iv,1336076765,2012-05-03 20:26:05+00:00,I'm also in this ballpark (300-600€ / day) whi...,3924840,None,None,0
7,3107534,Iv,Iv,1318520044,2011-10-13 15:34:04+00:00,how do you run unity in non-accelerated mode ?...,3107241,None,None,0
8,8409259,Iv,Iv,1412421647,2014-10-04 11:20:47+00:00,Polio is not exterminated even if it is absent...,8409226,None,None,0
9,2855741,Jd,Jd,1312690646,2011-08-07 04:17:26+00:00,"Yep, I didn't find Rescuetime very helpful. I ...",2855343,None,None,0


In [16]:
def execute_query(client, query):    
    # Set up the query (cancel the query if it would use too much of 
    # your quota, with the limit set to 10 GB)
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
    query_job = client.query(query, job_config=safe_config)

    # API request - run the query, and return a pandas DataFrame
    return query_job.to_dataframe()
    

### 2.1 Prolific commenters
Hacker News would like to send awards to everyone who has written more than 10,000 posts. Write a query that returns all authors with more than 10,000 posts as well as their post counts. Call the column with post counts NumPosts.



In [ ]:
query_1 = """ Select author, count(id) as NumPosts
from `bigquery-public-data.hacker_news.comments`
group by author 
having NumPosts > 10000"""
execute_query(query_1)

In [ ]:
df_results

We use **Having** to filter information after the groupby is done. We also can filter results before the groupby using **where** 

In [ ]:
query_2 = """ Select author, count(id) as NumPosts
from `bigquery-public-data.hacker_news.comments`
where CHAR_LENGTH(text) > 100
group by author 
having NumPosts > 10000"""

df_result_2 = execute_query(query_2)

In [ ]:
df_result_2

The remainded question is, Is posible to use **where** rahter than **having** to filter information after the **group by** command?

In [ ]:
query_3 = """ Select * from 
(Select author, count(id) as NumPosts
from `bigquery-public-data.hacker_news.comments`
where CHAR_LENGTH(text) > 100
group by author) as table_tmp 
where NumPosts > 10000"""

df_result_3 = execute_query(query_3)
df_result_3

### 2.2 Deleted comments
How many comments have been deleted? (If a comment was deleted, the deleted column in the comments table will have the value True.)

In [7]:
query_4 = """ Select deleted, count(id) as NumDeleted
from `bigquery-public-data.hacker_news.comments`
where deleted = True
group by deleted
"""
df_result_4 = execute_query(query_4)
df_result_4

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,deleted,NumDeleted
0,True,227736


In [10]:
query_5 = """ Select count(1) as NumDeleted
from `bigquery-public-data.hacker_news.comments`
where deleted = TRUE
group by deleted
"""

df_result_5 = execute_query(query_5)
df_result_5

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,NumDeleted
0,227736


# 3. Order By

In [12]:
def get_dataset_ref(client, project_bq, dataset_bq):    
    dataset_ref = client.dataset(dataset_bq, project=project_bq)    
    dataset = client.get_dataset(dataset_ref)
    return dataset_ref, dataset

def get_table_ref(client, dataset_ref, table_name):    
    table_ref = dataset_ref.table(table_name)    
    table = client.get_table(table_ref)
    return table_ref, table

In [19]:
# Create a "Client" object
client = bigquery.Client()
dataset_ref, dataset = get_dataset_ref(client, "bigquery-public-data", "world_bank_intl_education")
table_ref, table = get_table_ref(client, dataset_ref, "international_education")
df_head_table = client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  """


In [14]:
df_head_table

,country_name,country_code,indicator_name,indicator_code,value,year
0,Marshall Islands,MHL,Official entrance age to post-secondary non-te...,UIS.THAGE.4.A.GPV,1.900000e+01,2016
1,Mauritania,MRT,"GDP, PPP (constant 2011 international $)",NY.GDP.MKTP.PP.KD,1.536444e+10,2016
2,Mauritania,MRT,Population of the official age for tertiary ed...,SP.TER.TOTL.MA.IN,1.928930e+05,2016
3,"Micronesia, Fed. Sts.",FSM,Theoretical duration of post-secondary non-ter...,UIS.THDUR.4.A.GPV,2.000000e+00,2016
4,Montenegro,MNE,Population of the official age for pre-primary...,SP.PRE.TOTL.FE.IN,1.118200e+04,2016


### 3.1 Government expenditure on education
The value in the indicator_code column describes what type of data is shown in a given row.
One interesting indicator code is SE.XPD.TOTL.GD.ZS, which corresponds to "Government expenditure on education as % of GDP (%)".

Which countries spend the largest fraction of GDP on education?

To answer this question, consider only the rows in the dataset corresponding to indicator code SE.XPD.TOTL.GD.ZS, and write a query that returns the average value in the value column for each country in the dataset between the years 2010-2017 (including 2010 and 2017 in the average).

Requirements:

Your results should have the country name rather than the country code. You will have one row for each country.
The aggregate function for average is AVG(). Use the name avg_ed_spending_pct for the column created by this aggregation.
Order the results so the countries that spend the largest fraction of GDP on education show up first.
In case it's useful to see a sample query, here's a query you saw in the tutorial (using a different dataset):

#### Query to find out the number of accidents for each day of the week
```
query = """
        SELECT COUNT(consecutive_number) AS num_accidents, 
               EXTRACT(DAYOFWEEK FROM timestamp_of_crash) AS day_of_week
        FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2015`
        GROUP BY day_of_week
        ORDER BY num_accidents DESC
        """
```

In [26]:
query_6 = """
        SELECT country_code, country_name, avg(value) as avg_ed_spending_pct             
        FROM `bigquery-public-data.world_bank_intl_education.international_education` 
        where indicator_code = 'SE.XPD.TOTL.GD.ZS' and year>=2010 and year<=2017
        group by country_code, country_name
        order by avg_ed_spending_pct desc
        """
        
df_result_6 = execute_query(client, query_6)
df_result_6    
#     where indicator_code = 'SE.XPD.TOTL.GD.ZS' and year between 2010 and 2017

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,country_code,country_name,avg_ed_spending_pct
0,CUB,Cuba,12.837270
1,FSM,"Micronesia, Fed. Sts.",12.467750
2,SLB,Solomon Islands,10.001080
3,MDA,Moldova,8.372153
4,NAM,Namibia,8.349610
...,...,...,...
152,KHM,Cambodia,1.706404
153,PSE,West Bank and Gaza,1.503760
154,SSD,South Sudan,1.409726
155,MCO,Monaco,1.409606


In [27]:
query_6 = """
        SELECT country_code, country_name, avg(value) as avg_ed_spending_pct            
        FROM `bigquery-public-data.world_bank_intl_education.international_education` 
        where indicator_code = 'SE.XPD.TOTL.GD.ZS' and year between 2010 and 2017
        group by country_code, country_name
        order by avg_ed_spending_pct desc
        """
        
df_result_6 = execute_query(client, query_6)
df_result_6    

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,country_code,country_name,avg_ed_spending_pct
0,CUB,Cuba,12.837270
1,FSM,"Micronesia, Fed. Sts.",12.467750
2,SLB,Solomon Islands,10.001080
3,MDA,Moldova,8.372153
4,NAM,Namibia,8.349610
...,...,...,...
152,KHM,Cambodia,1.706404
153,PSE,West Bank and Gaza,1.503760
154,SSD,South Sudan,1.409726
155,MCO,Monaco,1.409606


### 3.2 Identify interesting codes to explore
The last question started by telling you to focus on rows with the code SE.XPD.TOTL.GD.ZS. But how would you find more interesting indicator codes to explore?

There are 1000s of codes in the dataset, so it would be time consuming to review them all. But many codes are available for only a few countries. When browsing the options for different codes, you might restrict yourself to codes that are reported by many countries.

Write a query below that selects the indicator code and indicator name for all codes with at least 175 rows in the year 2016.

Requirements:

* You should have one row for each indicator code.
* The columns in your results should be called indicator_code, indicator_name, and num_rows.
* Only select codes with 175 or more rows in the raw database (exactly 175 rows would be included).
* To get both the indicator_code and indicator_name in your resulting DataFrame, you need to include both in your SELECT statement (in addition to a COUNT() aggregation). This requires you to include both in your GROUP BY clause.
* Order from results most frequent to least frequent.

In [30]:
query_7 = """
        select * from 
        (SELECT indicator_code, indicator_name, count(1) as num_rows            
        FROM `bigquery-public-data.world_bank_intl_education.international_education` 
        where year = 2016
        group by indicator_code, indicator_name) as table_tmp
        where num_rows >= 175
        order by num_rows desc
        """
        
df_result_7 = execute_query(client, query_7)
df_result_7    

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,indicator_code,indicator_name,num_rows
0,SP.POP.GROW,Population growth (annual %),232
1,SP.POP.TOTL,"Population, total",232
2,IT.NET.USER.P2,Internet users (per 100 people),223
3,SH.DYN.MORT,"Mortality rate, under-5 (per 1,000)",213
4,SP.POP.1564.FE.IN,"Population, ages 15-64, female",213
5,SP.POP.TOTL.FE.ZS,"Population, female (% of total)",213
6,SP.POP.TOTL.MA.IN,"Population, male",213
7,SP.POP.0014.FE.IN,"Population, ages 0-14, female",213
8,SP.POP.1564.MA.IN,"Population, ages 15-64, male",213
9,SP.POP.1564.TO.ZS,"Population, ages 15-64 (% of total)",213


#### To avoid to use the second **where** creating and an additional table as the above, we can use **havig** as follow:

In [31]:
query_7 = """        
        SELECT indicator_code, indicator_name, count(1) as num_rows            
        FROM `bigquery-public-data.world_bank_intl_education.international_education` 
        where year = 2016
        group by indicator_code, indicator_name
        having num_rows >= 175
        order by num_rows desc
        """
        
df_result_7 = execute_query(client, query_7)
df_result_7    

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,indicator_code,indicator_name,num_rows
0,SP.POP.GROW,Population growth (annual %),232
1,SP.POP.TOTL,"Population, total",232
2,IT.NET.USER.P2,Internet users (per 100 people),223
3,SP.POP.TOTL.MA.IN,"Population, male",213
4,SP.POP.1564.TO,"Population, ages 15-64, total",213
5,SP.POP.0014.TO,"Population, ages 0-14, total",213
6,SP.POP.1564.MA.IN,"Population, ages 15-64, male",213
7,SP.POP.TOTL.FE.ZS,"Population, female (% of total)",213
8,SP.POP.1564.FE.IN,"Population, ages 15-64, female",213
9,SP.POP.TOTL.FE.IN,"Population, female",213
